In [ ]:
%matplotlib qt
%gui qt5
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as op
import csv
import matplotlib.pyplot as plt
import mne
from mne.decoding import SlidingEstimator
from sklearn.utils import shuffle
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
import batch_average as ba

### Importing preprocessed data - epochs

In [ ]:
overwrite = True
eeg_path = r'D:\Master_thesis\Pre-processed_data'
data_path_out = r'D:\Master_thesis\Figures\Multiclass\sterovsmono'
data_path_out_scores = r'D:\Master_thesis\Scores\Multiclass\sterovsmono'

# Multiclass

In [ ]:
l1_ratio = [0.05]
C = [0.1, 0.01]
scoring = 'accuracy' 


n_split= np.arange(1, 6, dtype=int)
# n_split= [1]
# subjects = ['VR2FEM_S01', 'VR2FEM_S02', 'VR2FEM_S03', 'VR2FEM_S04', 'VR2FEM_S05','VR2FEM_S06','VR2FEM_S07','VR2FEM_S08','VR2FEM_S10','VR2FEM_S11','VR2FEM_S12', 'VR2FEM_S14', 'VR2FEM_S15', 'VR2FEM_S17', 'VR2FEM_S18', 'VR2FEM_S21']


# subjects = ['VR2FEM_S18', 'VR2FEM_S02', 'VR2FEM_S03', 'VR2FEM_S04']
viewing_conditions = [ "stereo/neutral","stereo/happy", "stereo/angry", "stereo/surprised"], \
                    [ "mono/neutral","mono/happy", "mono/angry", "mono/surprised"]

for subject in os.listdir(eeg_path):
    data_path = os.path.join(eeg_path, subject)
    #reading preprocessed data -epochs
    
    epochs_train_list = []
    epochs_test_list = []
    
    for items in n_split:

        epochs_train_name = os.path.join(f"{data_path}\{subject}-preprocessed_train_{items}-epo.fif")
        epochs_test_name = os.path.join(f"{data_path}\{subject}-preprocessed_test_{items}-epo.fif")
        epochs_train = mne.read_epochs(epochs_train_name, preload=True)
        epochs_test = mne.read_epochs(epochs_test_name, preload=True)
        epochs_train_list.append(epochs_train)
        epochs_test_list.append(epochs_test)
    for view in viewing_conditions:
        print(f"Viewing Condition: {view}")
        viewing_type = view[0].split("/")[0]

        scores_avg_all = []
        predictions_all = []
        y_test_all = []

        for l1 in l1_ratio:
            for cs in C:
                print("Running reg. ratio for: ",l1)
                print("Running reg. strength for: ",cs)   
                clf = make_pipeline(
                StandardScaler(),
                LogisticRegression(multi_class='ovr', max_iter = 4000, solver='saga',penalty='elasticnet', l1_ratio=l1, C=cs))

                time_decoder = SlidingEstimator(clf, scoring=scoring, n_jobs=1, verbose=True)

                print("----------------Start Split ----------------")

                scores_all_mini = []

                for split in n_split:
                    print(f"Subject: {subject} Number of test/train split: {split} for L1 ratio : {l1} and strength: {cs}")
                    n = split-1
                    x_train = epochs_train_list[n][view].get_data()
                    events_train = epochs_train_list[n][view].events[:,2]
                    y_train = [int(str(yy)[-1]) for yy in events_train]

                    x_test = epochs_test_list[n][view].get_data()
                    events_test = epochs_test_list[n][view].events[:,2]
                    y_test = [int(str(yy)[-1]) for yy in events_test] 

                    time_decoder.fit(x_train, y_train)
                    predictions = time_decoder.predict(x_test)
                    scores_mini = time_decoder.score(x_test, y_test)
                    scores_all_mini.append([scores_mini[:]])

                print("----------------Ending Split ----------------")
                scores_CV_mini = np.vstack(scores_all_mini)
                scores_avg_mini = np.mean(scores_CV_mini, axis=0)
                print("----------------Running visuals ----------------")   

                dir_save= os.path.join(data_path_out_scores, subject)

                if not op.exists(dir_save):
                        os.makedirs(dir_save)
                score_file_name = f"{dir_save}\{subject}_multiclass_decoding_L1_ratio {l1}_C_{cs}_{viewing_type}.csv"
                print(score_file_name)
                with open(score_file_name, "w") as file:
                    writer = csv.writer(file)
                    writer.writerow([scores_avg_mini])

                fig, ax = plt.subplots()
                ax.plot(epochs_train.times, scores_avg_mini, label='score')
                ax.axhline(.25, color='k', linestyle='--', label='chance')
                ax.axvline(.0, color='k', linestyle='-')
                ax.set_xlabel('Times')
                ax.set_ylabel('Accuracy')  
                ax.legend()
                ax.axvline(.0, color='k', linestyle='-')
                ax.set_title(f"Multiclass Decoding - Stereo vs Mono")


                dir_save= os.path.join(data_path_out, subject)
                if not op.exists(dir_save):
                            os.makedirs(dir_save)
                overwrite = True
                resolution_value = 1200
                plt.savefig(f"{dir_save}\{subject}_multiclass_decoding_L1_ratio_{l1}_C_{cs}_{viewing_type}.jpg", dpi=resolution_value, overwrite= overwrite)
